# Banglapedia full scraping

In [37]:
import os
import requests
from bs4 import BeautifulSoup as bs
import time
from concurrent.futures import ThreadPoolExecutor

In [39]:
source = requests.get("https://bn.banglapedia.org/index.php?")
soup = bs(source.text, 'lxml')
#print(soup.prettify())

In [33]:
letter_table = soup.find('div', class_='table-responsive')
#print(letter_table.prettify())

In [44]:
# Find all table cells with links
table_cells = letter_table.find_all('td')

# Extract text and href attributes
for cell in table_cells:
    link = cell.find('a')  # Find the <a> tag inside the <td>
    if link:
        #text = link.get_text(strip=True)  # Get the text inside the <a>
        href = link['href']  # Get the href attribute of the <a>
        href = f"https://bn.banglapedia.org{href}"
        #print(f"{text}")
        #print(href)

In [46]:
# Function to process a single link
def process_link(link_data):
    text, full_link, session = link_data
    try:
        # Fetch content for the link using the session
        response = session.get(full_link)
        content_soup = bs(response.text, 'lxml')

        # Extract the text content
        text_content = content_soup.find('div', class_='mw-parser-output').get_text(separator='\n', strip=True)

        # Save to a file
        file_path = f"banglapedia/{text}.txt"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure the directory exists
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(text_content)

        return True  # Indicates success
    except Exception as e:
        print(f"Error processing link {full_link}: {e}")
        return False  # Indicates failure

# Main script
source = requests.get("https://bn.banglapedia.org/index.php?")
soup = bs(source.text, 'lxml')
letter_table = soup.find('div', class_='table-responsive')

# Find all table cells with links
table_cells = letter_table.find_all('td')

# Extract text and href attributes
links_data = []
for cell in table_cells:
    link = cell.find('a')  # Find the <a> tag inside the <td>
    if link:
        #text = link.get_text(strip=True)  # Get the text inside the <a>
        href = link['href']  # Get the href attribute of the <a>        
        href = f"https://bn.banglapedia.org{href}"
        #print(f"{text}")
        #print(href)
        
        source1 = requests.get(href)
        soup1 = bs(source1.text, 'lxml')

        container = soup1.find('div', class_='mw-body-content')
        link_container = container.find_all('li')

        # Extract text and href attributes
        for url in link_container:
            link = url.find('a')  # Find the <a> tag inside the <li>
            if link:
                text = link.get_text(strip=True)  # Get the text inside the <a>
                href = link['href']  # Get the href attribute of the <a>
                full_link = f"https://bn.banglapedia.org{href}"
                links_data.append((text, full_link))

start_time = time.time()

# Use ThreadPoolExecutor with a shared requests.Session
with requests.Session() as session:
    with ThreadPoolExecutor(max_workers=50) as executor:
        # Include the session in the link data
        results = list(executor.map(process_link, [(text, full_link, session) for text, full_link in links_data]))

# Calculate the total links processed successfully
successful_links = sum(results)
duration = time.time() - start_time

print(f"Successfully scraped {successful_links} out of {len(links_data)} links in {duration:.2f} seconds")
print(f"Collected {len(links_data)} links so far.")

Error processing link https://bn.banglapedia.org/index.php?title=%E0%A6%95%E0%A6%B0: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))
Successfully scraped 14416 out of 14417 links in 1008.19 seconds
Collected 14417 links so far.


In [ ]:
# Without using session and multithreading
  #--> Scraped 345 links of content in 322.02347707748413 seconds
# Using only session 
  #-->Scraped 345 links of content in 327.3671143054962 seconds
# Using only ThreadPoolExecutor
  #-->Successfully scraped 345 out of 345 links in 114.21 seconds
# Using ThreadPoolExecutor and session
  #-->Successfully scraped 345 out of 345 links in 16.67 seconds

In [ ]:
source1 = requests.get("https://bn.banglapedia.org/index.php?title=বিশেষ:সব_পাতা/অ")   
soup1 = bs(source1.text, 'lxml')
#rint(soup1.prettify())

container= soup1.find('div', class_='mw-body-content')
#print(link_container.prettify())

start_time = time.time()

link_container = container.find_all('li')

total_links = 0
# Extract text and href attributes
for url in link_container:
    link = url.find('a')  # Find the <a> tag inside the <td>
    if link:
        text = link.get_text(strip=True)  # Get the text inside the <a>
        href = link['href']  # Get the href attribute of the <a>
        full_link = f"https://bn.banglapedia.org{href}"
        total_links += 1 
        #print(f"{text}")
        #print(f"{full_link}")
#print(f"Total links scraped: {total_links}")
       
        content_page = requests.get(full_link)
        content_soup =  bs(content_page.text, 'lxml')
        #print(content_soup.prettify())
        text_content = content_soup.find('div', class_='mw-parser-output').get_text(separator='\n', strip=True)
        #print(text_content)

        file_path = f"banglapedia/{text}.txt"
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(text_content)
            
duration = time.time() - start_time        
print(f"Scraped {total_links} links of content in {duration} seconds")        

In [ ]:
# Function to process a single link
def process_link(link_data):
    
    text, full_link, session = link_data
    try:
        # Fetch content for the link using the session
        response = session.get(full_link)
        content_soup = bs(response.text, 'lxml')

        # Extract the text content
        text_content = content_soup.find('div', class_='mw-parser-output').get_text(separator='\n', strip=True)

        # Save to a file
        file_path = f"banglapedia/{text}.txt"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure the directory exists
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(text_content)

        return True  # Indicates success
    except Exception as e:
        print(f"Error processing link {full_link}: {e}")
        return False  # Indicates failure

# Main script
source1 = requests.get("https://bn.banglapedia.org/index.php?title=বিশেষ:সব_পাতা/অ")
soup1 = bs(source1.text, 'lxml')

container = soup1.find('div', class_='mw-body-content')
link_container = container.find_all('li')

# Extract text and href attributes
links_data = []
for url in link_container:
    link = url.find('a')  # Find the <a> tag inside the <li>
    if link:
        text = link.get_text(strip=True)  # Get the text inside the <a>
        href = link['href']  # Get the href attribute of the <a>
        full_link = f"https://bn.banglapedia.org{href}"
        links_data.append((text, full_link))

start_time = time.time()

# Use ThreadPoolExecutor with a shared requests.Session
with requests.Session() as session:
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Include the session in the link data
        results = list(executor.map(process_link, [(text, full_link, session) for text, full_link in links_data]))

# Calculate the total links processed successfully
successful_links = sum(results)
duration = time.time() - start_time

print(f"Successfully scraped {successful_links} out of {len(links_data)} links in {duration:.2f} seconds")
